# Titanic Survival Classification
Based on the notebooks by:  
[Alexis Cook](http://www.kaggle.com/alexisbcook/titanic-tutorial)  
[Francisco Javier Gallego](https://www.kaggle.com/javigallego/top-5-hyperparameter-tuning-ensemble-modeling?scriptVersionId=88795500)  
[Woo Seung Han](https://www.kaggle.com/hadeux/titanic-survivor-predict-eda-lightgbm-kor-eng/notebook)  

In [ ]:
# autocomplete ON
%config Completer.use_jedi = False

In [ ]:
# Load libraries
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.stats as stats

import seaborn as sns
import matplotlib.pyplot as plt

## <b> 1 <span style='color:#ffb2ae'> | </span> Data prep & EDA </b>

<div style='color: white; display: fill; 
    border-radius: 8px; background-color: #ffb2ae;
    font-size: 100%; letter-spacing:0.5px'> 
    <p style='padding: 8px; color: black;'> 
        <b>1.1 &nbsp; Load data </b>
    </p>
</div> 

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Load train and test data
df_train = pd.read_csv('../input/titanic/train.csv')
df_test = pd.read_csv('../input/titanic/test.csv')
gender_submission = pd.read_csv('../input/titanic/gender_submission.csv')

# Concatenate two data sets
df_data = pd.concat([df_train, df_test], sort=True).reset_index(drop=True)

<div style='color: white; display: fill; 
    border-radius: 8px; background-color: #ffb2ae;
    font-size: 100%; letter-spacing:0.5px'> 
    <p style='padding: 8px; color: black;'> 
        <b>1.2 &nbsp; EDA </b>
    </p>
</div> 

In [ ]:
## Visually inspect first few rows of each table
df_train.head(3)
# df_test.head(3)
# gender_submission.head(3)

### Task is to predict "Survived" for each "PassengerId"

In [ ]:
## Show table schema
# df_train.info()
# df_test.info()
df_data.info()

### Excluding the Id and independent variable, there are 10 features

In [ ]:
df_data.describe(include='all')

### Seems to be duplicate names and tickets
### Overall survival rate is 38.38%

In [ ]:
# Check for any duplicates
df_data.duplicated().value_counts()

### No duplicate rows between training and test datasets

# Check for people with the same names
# df_data.Name.duplicated().value_counts()
# df_train[df_train['Name'].isin(df_data[df_data.Name.duplicated()].Name.tolist())]
# df_test[df_test['Name'].isin(df_data[df_data.Name.duplicated()].Name.tolist())]
df_data[df_data['Name'].isin(df_data[df_data.Name.duplicated()].Name.tolist())]

### There are two pairs of people exactly matching the names Miss Kate Connolly and Mr. Kelly James respectively.
### Upon inspection, people in both pairs have different passenger ID, ticket number, and paid different a different fare amount.
### It should be safe to assume that they are all different people with the same name. 

In [ ]:
## Count missing values from each column
# df_train.isna().sum()
df_test.isna().sum()
# df_data.isna().sum()

### Columns "Age", "Cabin", and "Embarked" have missing values

In [ ]:
# Functions for drawing pair plots with annotations

def corrfunc(x, y, **kws):
    r, _ = stats.pearsonr(x, y)
    ax = plt.gca()
    ax.annotate("r = {:.2f}".format(r),
                xy=(.1, .9), xycoords=ax.transAxes)
    
def pairPlot(data, column=None):
    if column:
        num = len(data[column].unique())
        g = sns.PairGrid(data, hue=column, hue_kws=sns.color_palette('vlag', n_colors=2, as_cmap=True), dropna=True)
        g.map_offdiag(sns.scatterplot)
        g.add_legend()
    else:
        g = sns.PairGrid(data, dropna=True)
        g.map_offdiag(corrfunc)
        g.map_offdiag(sns.regplot)
    g.map_diag(sns.histplot)
    return g

In [ ]:
# Visualize pair-wise distribution of numerical features using a pair plot
pairPlot(df_data)
pairPlot(df_data, 'Survived')
# sns.pairplot(data=df_data, kind = 'reg')
# sns.pairplot(data=df_data, kind = 'hist', hue='Survived', palette={0:'blue',1:'green'})

In [ ]:
# Heat map to visualize correlation between variables
sns.heatmap(df_data.corr(), annot=True, cmap='vlag')

In [ ]:
# Tally groups to see which features may be important for survival
# Especially categorical variables not visualized by pair plots above

# df_train.groupby(['Survived', 'Sex']).size()
def categoricalPlots(data, y=None):
    for col in data:
        if col == y:
            continue
        elif (len(data[col].unique())) < 10:
            x=col
            (data
            .groupby(x)[y]
            .value_counts(normalize=True)
            .mul(100)
            .rename('Percent')
            .reset_index()
            .pipe((sns.barplot, 'data'), x=x, y='Percent', hue=y))
            plt.show()
categoricalPlots(df_train, y='Survived')
# sns.countplot(x='Survived', hue='Sex', data = df_train)

#### **EDA KEY INSIGHTS**
1. There is class imbalance in the trainng set (more people died than survived)
2. There were duplicate names between train and test data sets but upon closer inspection, they seem to be different people with the same names. 
3. There are missing values for the following features: "Age", "Cabin", "Embarked", and "Fare". 
4. "Ticket" and "Cabin" are complex string variables that may be important indicator in predicting survival. 
5. Among numerical features, "Fare" is correlated with survival
6. All categorical features seem to be associated with survival, especially "Sex" and "PClass".  

## <b> 2 <span style='color:#ffc967'> | </span> Data processing</b>

<div style='color: white; display: fill; 
    border-radius: 8px; background-color: #ffc967;
    font-size: 100%; letter-spacing:0.5px'> 
    <p style='padding: 8px; color: black;'> 
        <b>2.1 &nbsp; Feature engineering</b>
    </p>
</div> 

#### Title (Honorific)
Some people have titles (ex. Mr. Mrs.) in their names. In addition to biological sex, their titles can signify positions of power and authority (ex. Capt, Countess, Sir, etc.) which may have impacted survival. 

In [ ]:
# Extract title (honorifics) from people's names
df_data['Title'] = df_data.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
# Count occurence for each title
pd.crosstab(df_data['Title'], df_data['Sex']).transpose()

In [ ]:
# Replace upper-class titles with 'Elite' status and unify titles for men and women
df_data['Title'] = df_data['Title'].replace(['Capt', 'Col', 'Countess', 'Dr', 'Jonkheer' ,'Lady', 'Major', 'Rev', 'Sir'], 'Elite')
df_data['Title'] = df_data['Title'].replace(['Ms', 'Mlle', 'Dona'], 'Miss')
df_data['Title'] = df_data['Title'].replace('Mme', 'Mrs')
df_data['Title'] = df_data['Title'].replace('Don', 'Mr')
# Count occurence for each title again
pd.crosstab(df_data['Title'], df_data['Sex']).transpose()

In [ ]:
# Calculating the mean survival rate for each title/class
df_data[['Title', 'Survived']].groupby(['Title'], as_index=False).mean().transpose()

#### Surnames
People with same surname may be relatives that boarded the ship together. 

In [ ]:
# Extract surnames from name
df_data['Surname'] = df_data.Name.str.extract('^([A-Za-z\'\-\s]+),', expand=False)
# Add new column of counts of people with same surnames
df_data['SurnameCount'] = df_data['Surname'].value_counts()[df_data['Surname']].values

#### Deck
Cabins are located on decks A-G. Which deck a passenger's cabin is located on is associated with "PClass" and can be predictive of survival. 

2208 people were aboard Titanic - 891 of which were crew [(source)](https://www.encyclopedia-titanica.org/titanic/#titanic-passengers-crew)  
Below are passenger capacities for each class on each deck [(source)](https://www.encyclopedia-titanica.org/passenger-accommodation.html) [(source)](https://titanic.fandom.com)

| DECK | FIRST CLASS | SECOND CLASS | THIRD CLASS |
| --- | --- | --- | --- |
| T | 7 | 0 | 0 |
| A | 42 | 0 | 0 |
| B | 123 | 0 | 0 |
| C | 310 | 0 | 0 |
| D | 117 | 118 | 50 |
| E | 97 | 226 | 260 |
| F | 0 | 218 | 466 |
| G | 0 | 112 | 250 |
| **TOTAL** | **689** | **674** | **1026** |

In [ ]:
df_data[df_data['Cabin'].notna()]['Cabin'].unique()

In [ ]:
df_data['Deck'] = df_data.Cabin.str.extract('^([A-Za-z])', expand=False)
print(df_data['Deck'].value_counts(dropna=False))
### There is one passenger on Deck T
### There are many missing values
pd.crosstab(df_data['Deck'], df_data['Pclass'], margins=True, dropna=False).transpose()

In [ ]:
# Do research on passenger that was on Deck T
df_data[df_data['Deck']=='T']
### It seems that Mr. Blackwell was on the very top deck above Deck A.
# df_data['Deck'] = df_data['Deck'].replace('T', 'A')

#### DistToBoat
On the Titanic, all life boats were on the Boat deck [(source)](https://titanic.fandom.com/wiki/Boat_Deck), which was the uppermost deck.  
The following is the arrangment of decks from top to bottom of the boat [(source)](https://www.encyclopedia-titanica.org/titanic-deckplans/) and their height to the next deck above [(source)](titanicinquiry.org/BOTInq/BOTReport/botRepTitanic.php):  
- Boat Deck
- A (Promenade) Deck: 9ft 6in (2.8956m)
- B (Bridge) Deck: 9ft (2.7432m)
- C (Shelter) Deck: 9ft (2.7432m)
- D (Saloon) Deck: 10ft 6in (3.2004m)
- E (Upper) Deck: 9ft (2.7432m)
- F (Middle) Deck: 8ft 6in (2.5908m)
- G (Lower) Deck: 8ft (2.4384m)
- Orlop Deck: 8ft (2.4384m)

G deck was just above the waterline and started flooding within 15 minutes of collision. [(source)](https://titanic.fandom.com/wiki/G_Deck)  
Therefore, higher passengers are, more closer they are to the life boats and more likely they are to survive. 
Instead of using "Cabin" and "Deck" features, they will be converted to "DistToBoat" feature - the height the passengers need to climb to get to the Boat Deck. 

In [ ]:
df_data['DistToBoat'] = df_data['Deck'].replace({'T':0, 'A':2.8956, 'B':2.8956+2.7432, 'C':2.8956+2.7432+2.7432,
                                                 'D':2.8956+2.7432+2.7432+3.2004, 'E':2.8956+2.7432+2.7432+3.2004+2.7432,
                                                 'F':2.8956+2.7432+2.7432+3.2004+2.7432+2.5908,
                                                 'G':2.8956+2.7432+2.7432+3.2004+2.7432+2.5908+2.4384})
df_data['DistToBoat'].describe()

#### Family size
"Parch" and "SibSp" features can be summed to a new feature: "FamilySize"

In [ ]:
# Create new feature: "FamilySize"
df_data['FamilySize'] = df_data['Parch'] + df_data['SibSp'] + 1

"FamilySize" does not account for number of cousins or distant relatives.  
"SurnameCount" may be better indicator for family size

In [ ]:
# Inspect rows that have different values for "FamilySize" and "SurnameCount"
df_data[df_data['FamilySize']-df_data['SurnameCount'] != 0][['Cabin','Embarked','Name','Ticket','Surname','SurnameCount','FamilySize']].sort_values(by='Surname').head(19)


Most people that have same surname have similar ticket numbers and embarked from the same port.  
We can see that counting people with the same surname is better indicator of family size.  
In the future, taking "Ticket" and "Embarked" variables into account may help identify people who are not family members that are travelling together.  

#### Pclass
Will change PClass to categorical variable

In [ ]:
df_data['Pclass'] = df_data['Pclass'].astype('category')

<div style='color: white; display: fill; 
    border-radius: 8px; background-color: #ffc967;
    font-size: 100%; letter-spacing:0.5px'> 
    <p style='padding: 8px; color: black;'> 
        <b>2.2 &nbsp; Imputation of missing values</b>
    </p>
</div> 

As discovered from EDA, there are missing values for the following features: "Age", "Cabin", "Embarked", and "Fare". We will address each feature one-by-one. 

#### Age
As previously discovered in EDA, age is most highly correlated with "Pclass" feature. 
Age of passengers may also be associated with biological "Sex". 

In [ ]:
# Plot age for each "Pclass" and "Sex"
sns.violinplot(x=df_data.Pclass.astype('category'), y=df_data.Age, hue=df_data.Sex, split=True)

### Differences in age distribution for different "Pclass" and "Sex" could be observed. 

In [ ]:
df_data.Age.describe()

In [ ]:
# Check median age for each subgroup
print(df_data.groupby(['Pclass','Sex']).agg('median').Age)

print("\nNumber of NAs for Age is (before): ", df_data.Age.isna().sum())

# Fill NaN values with median value for each subgroups
df_data['Age'] = df_data.groupby(['Pclass','Sex']).Age.apply(lambda x: x.fillna(x.median()))

print("\nNumber of NAs for Age is (after): ", df_data.Age.isna().sum())

#### Cabin/Deck/DistToBoat
Instead of assigning the most frequent value or separate 'missing' designation for missing values in Cabin or Deck feature, we can assign the mean "DistToBoat" for each "Pclass" since we know for each passenger class, how many rooms are allocated on each deck.  

| DECK | HeightToBoat (m) | FIRST CLASS | SECOND CLASS | THIRD CLASS |
| --- | --- | --- | --- | --- |
| T | 0 | 7 | 0 | 0 |
| A | 2.8956 | 42 | 0 | 0 |
| B | 5.6388 | 123 | 0 | 0 |
| C | 8.382 | 310 | 0 | 0 |
| D | 11.5824 | 117 | 118 | 50 |
| E | 14.3256 | 97 | 226 | 260 |
| F | 16.9164 | 0 | 218 | 466 |
| G | 19.3548 | 0 | 112 | 250 |
| **TOTAL** |  | **689** | **674** | **1026** |

In [ ]:
meanDistToBoat = {'1':(7*0 + 42*2.8956 + 123*5.6388 + 310*8.382 + 117*11.5824 + 97*14.3256)/(689),
                  '2':(118*11.5824 + 226*14.3256 + 218*16.9164 + 112*19.3548)/(674),
                  '3':(50*11.5824 + 260*14.3256 + 466*16.9164 + 250*19.3548)/(1026)}
meanDistToBoat

In [ ]:
df_data['DistToBoat'] = df_data.apply(lambda x: meanDistToBoat[str(x.Pclass)] if pd.isna(x.DistToBoat) else x.DistToBoat, axis=1)

#### Embarked
"Embarked" feature tells us which of the 3 ports (C = Cherbourg; Q = Queenstown; S = Southamptom) the passenger boarded the Titanic. There are two missing values for this feature.  
We could fill it in with the most frequent value (S) or better yet, we could look where people with the same surname boarded the ship. 

In [ ]:
# Look for the passengers with missing "Embarked" feature

# df_data[df_data.Embarked.isnull()]
# df_data[df_data.Surname=='Icard']
# df_data[df_data.Surname=='Stone']
df_data[df_data.Ticket=='113572']

### Interestingly, both have the same ticket number, boarded the ship alone.

In [ ]:
# Since we cannot find anyone related to the two passengers (except each other), fill the missing value with the mode. 
modeEmbarked = df_data['Embarked'].mode()[0]
df_data['Embarked'] = df_data['Embarked'].replace(np.nan, modeEmbarked)

#### Fare
There is one missing value for "Fare". We can simply fill this in with the mean fare from the same "Pclass" and "Embarked" group. 

In [ ]:
df_data[df_data.Fare.isnull()]

In [ ]:
df_data.groupby(['Pclass','Embarked']).agg('mean').Fare

In [ ]:
df_data['Fare'] = df_data['Fare'].fillna(14.435422)

<div style='color: white; display: fill; 
    border-radius: 8px; background-color: #ffc967;
    font-size: 100%; letter-spacing:0.5px'> 
    <p style='padding: 8px; color: black;'> 
        <b>2.3 &nbsp; Split data</b>
    </p>
</div> 
Since we merged training and validation data in the beginning, they will be split back for modeling. 

In [ ]:
df_train = df_data[df_data.Survived.notna()]
df_test = df_data[df_data.Survived.isna()]

## <b> 3 <span style='color:#90ee90'> | </span> Modeling</b>

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier

<div style='color: white; display: fill; 
    border-radius: 8px; background-color: #90ee90;
    font-size: 100%; letter-spacing:0.5px'> 
    <p style='padding: 8px; color: black;'> 
        <b>3.1 &nbsp; Feature selection & Encoding</b>
    </p>
</div> 

In [ ]:
df_train.head(2)

In [ ]:
# Train set
X_train = df_train[['Age', 'Embarked','Pclass','Sex','Title','SurnameCount','DistToBoat','FamilySize']]
X_train = pd.get_dummies(X_train)
Y_train = df_train['Survived'].astype('int64')

# Test set
X_test = df_test[['Age', 'Embarked','Pclass','Sex','Title','SurnameCount','DistToBoat','FamilySize']]
X_test = pd.get_dummies(X_test)

<div style='color: white; display: fill; 
    border-radius: 8px; background-color: #90ee90;
    font-size: 100%; letter-spacing:0.5px'> 
    <p style='padding: 8px; color: black;'> 
        <b>3.2 &nbsp; K-fold cross-validation</b>
    </p>
</div> 

In [ ]:
# Prepare to split the training set into 10 folds
kfold = StratifiedKFold(n_splits=10)

#### Random Forest Classifier
Mean cross-validation accuracy: 0.8283

In [ ]:
RFC = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=7)
RFC_cvs = cross_val_score(RFC, X=X_train, y=Y_train, scoring='accuracy', cv=kfold, n_jobs=-1)

In [ ]:
print(RFC_cvs)
RFC_cvs.mean()

#### Extremely Randomized Trees Classifier
Mean cross-validation accuracy: 0.8249

In [ ]:
ETC = ExtraTreesClassifier(n_estimators=100, max_depth=5, random_state=7)
ETC_cvs = cross_val_score(ETC, X=X_train, y=Y_train, scoring='accuracy', cv=kfold, n_jobs=-1)

In [ ]:
print(ETC_cvs)
ETC_cvs.mean()

#### AdaBoost
Mean cross-validation accuracy: 0.8271

In [ ]:
ABC = AdaBoostClassifier(n_estimators=100, random_state=7)
ABC_cvs = cross_val_score(ABC, X=X_train, y=Y_train, scoring='accuracy', cv=kfold, n_jobs=-1)

In [ ]:
print(ABC_cvs)
ABC_cvs.mean()

#### Gradient Boosting Classifier
Mean cross-validation accuracy: 0.8204

In [ ]:
GBC = GradientBoostingClassifier(n_estimators=100, max_depth=5, random_state=7)
GBC_cvs = cross_val_score(GBC, X=X_train, y=Y_train, scoring='accuracy', cv=kfold, n_jobs=-1)

In [ ]:
print(GBC_cvs)
GBC_cvs.mean()

<div style='color: white; display: fill; 
    border-radius: 8px; background-color: #90ee90;
    font-size: 100%; letter-spacing:0.5px'> 
    <p style='padding: 8px; color: black;'> 
        <b>3.5 &nbsp; Random hyperparameter grid search</b>
    </p>
</div> 

[(source)](https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74)

In [ ]:
def RandomGridSearch(model, grid, x, y, folds=3):
    fold = StratifiedKFold(n_splits=folds)
    # Do a random hyperparameter grid search
    random_model = RandomizedSearchCV(estimator=model, 
                                    param_distributions=grid, 
                                    n_iter=100, cv=fold, verbose=1,
                                    n_jobs=-1)
    random_model.fit(x, y)
    print('Best parameters: \n', random_model.best_params_)
    print('Best estimator: \n', random_model.best_estimator_)
    print('Best score: \n', random_model.best_score_)
    print('CV results: ')
    print('min_score: ', min(random_model.cv_results_['mean_test_score']))
    print('max_score: ', max(random_model.cv_results_['mean_test_score']))
    return random_model

#### Gradient Boosting Classifier
Maximum Public Score Achieved: 0.76794

In [ ]:
# Loss function to be optimized: {'deviance','exponential'}
loss = ['deviance']

# Number of boosting stages to perform
n_estimators = [100,300,600,900]

# Shrinks contribution of each tree - trade-off with n_estimators
learning_rate = [0.01, 0.05, 0.1]

# limits the number of nodes in tree
max_depth = [3,4,5,6,7,8]

# Min number of samples required to split an internal node
min_samples_split = [2, 5, 10]

# Min number of samples required to be at a leaf node (smoothing)
min_samples_leaf = [1, 10, 100]

# Number of features to consider when looking for the best split
max_features = [None, 'sqrt']

In [ ]:
random_grid = {'loss':loss,
              'n_estimators':n_estimators,
              'learning_rate':learning_rate,
              'max_depth':max_depth,
              'min_samples_split':min_samples_split,
              'min_samples_leaf':min_samples_leaf,
              'max_features':max_features}

In [ ]:
# %%time
# GBC = GradientBoostingClassifier(random_state=7)
# GBC_random = RandomGridSearch(GBC, random_grid, X_train, Y_train, folds=10)

In [ ]:
# predictions = GBC_random.best_estimator_.predict(X_test)

# output = pd.DataFrame({'PassengerId': df_test.PassengerId, 
#                       'Survived': predictions.astype('int64')})
# output.to_csv('submission.csv', index=False)
# print('Your submission was successfully saved!')

#### Random Forest Classifier


In [ ]:
# Number of boosting stages to perform
n_estimators = [100,300,600,900]

# function to measure quality of a split (tree-specific)
criterion = ['gini', 'entropy']

# limits the number of nodes in tree
max_depth = [3, 5, 8, None]

# Min number of samples required to split an internal node
min_samples_split = [2, 5, 10]

# Min number of samples required to be at a leaf node (smoothing)
min_samples_leaf = [1, 10, 100]

# Number of features to consider when looking for the best split
max_features = [None, 'sqrt']

# Whether bootstrap samples are used when building trees
bootstrap = [True, False]

In [ ]:
random_grid = {'n_estimators':n_estimators,
              'criterion':criterion,
              'max_depth':max_depth,
              'min_samples_split':min_samples_split,
              'min_samples_leaf':min_samples_leaf,
              'max_features':max_features,
              'bootstrap': bootstrap}

In [ ]:
%%time
RFC = RandomForestClassifier(random_state=7)
RFC_random = RandomGridSearch(RFC, random_grid, X_train, Y_train)

In [ ]:
predictions = RFC_random.best_estimator_.predict(X_test)

output = pd.DataFrame({'PassengerId': df_test.PassengerId, 
                      'Survived': predictions.astype('int64')})
output.to_csv('submission.csv', index=False)
print('Your submission was successfully saved!')

#### Random Forest Classifier (hyperparameters from Alexis Cook)
Maximum Public Score Achieved: 0.78229 (without further optimization)

In [ ]:
# RFC = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=7)
# RFC.fit(X_train, Y_train)

# predictions = RFC.predict(X_test)

## <b> 4 <span style='color:#92a1cf'> | </span> Model Interpretation</b>

# ***Work in progress***